In [1]:
from IPython.display import display, HTML
display(HTML ("""
<style>
div.container{width:90% !important;}
div.cell.code_cell.rendered{width:100%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:12pt;}
div.text_cell_render.rendered_html{font-size:12pt;}""
div.output {font-size:12pt; font-weight:bold;}
div.input{font-family:Consolas; font-size:12pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:12pt;padding:5px;}
table.dataframe{font-size:12px;}
</style>
"""))

# 문장 -> 임베딩 벡터(1차원 숫자 배열)
- openAI API의 키 (text-embedding-3-large)를 .env에 추가
- upstage(https://console.upstage.ai) 의 키 UPSTAGE_API_KEY를 .env에 추가
# 1. 환경변수 load

In [4]:
from dotenv import load_dotenv

import os
load_dotenv(
   # dotenv_path='.env'
)

openai_key = os.getenv('OPENAI_API_KEY')
upstage_key = os.getenv('UPSTAGE_API_KEY')
print(openai_key[:2])
print(upstage_key[:2])

sk
up


# 2. 유사도 계산하는 방법
- 1. 유클리드 거리 : 두 벡터간 거리가 가까운지
- 2. 코사인 유사도 : 두 벡터간 방향이 유사한지
- 3. dot product : 두 벡터간 곱을 사용하여 거리와 방향을 모두 고려

In [6]:
import numpy as np
def cosine_similarity(vec1, vec2):
    """두 백터 사이의 코사인 유사도 계산"""
    dot_product = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1) 
    norm_vec2 = np.linalg.norm(vec2)
    if norm_vec1==0 or norm_vec2==0:
        return 0.0
    return dot_product / (norm_vec1*norm_vec2)

# 3. openAI API의 embedding model 사용
- text-embedding-3-large

In [9]:
from openai import OpenAI
openai_client = OpenAI(
    # api_key=openai_key
)

In [10]:
# text-embedding-3-large(과금)
response = openai_client.embeddings.create(
    input = "The king is the price's father.",
    model = "text-embedding-3-large"
)

In [11]:
response.usage

Usage(prompt_tokens=8, total_tokens=8)

In [12]:
response.data[0].embedding

[0.0130026675760746,
 0.009061609394848347,
 -0.010121033526957035,
 0.04305500537157059,
 -0.001560885226354003,
 -0.05619186535477638,
 0.018829502165317535,
 -0.015735981985926628,
 -0.015072076581418514,
 0.012734279967844486,
 0.008433017879724503,
 0.010226976126432419,
 -0.030935190618038177,
 -0.005134676583111286,
 0.04992007464170456,
 -0.013391123153269291,
 -0.039664845913648605,
 0.006183506455272436,
 -0.055174820125103,
 -0.01736043393611908,
 -0.0020994257647544146,
 -0.02717776596546173,
 0.0130026675760746,
 -0.027290770784020424,
 -0.012162190861999989,
 -0.0031641472596675158,
 -0.03689621761441231,
 0.0037892076652497053,
 -0.018010213971138,
 0.009570132941007614,
 0.0007883882499299943,
 0.0024366758298128843,
 0.007038109004497528,
 0.012275196611881256,
 -0.01683778315782547,
 -0.020270319655537605,
 -0.023942990228533745,
 -0.012571834959089756,
 -0.016583522781729698,
 -0.02450801618397236,
 -0.024748152121901512,
 -0.03367556631565094,
 -0.03489037603139877,

In [16]:
king_vector = np.array(response.data[0].embedding) # The king is ~ 문장을 숫자로 바꾼 vector
print(king_vector.shape)
print(king_vector)

(3072,)
[ 0.01300267  0.00906161 -0.01012103 ... -0.01120871  0.02039745
  0.00075572]


In [18]:
queen_response = openai_client.embeddings.create(
    input = "The queen is the prince's mother",
    model = "text-embedding-3-large"
)

In [19]:
queen_response.usage

Usage(prompt_tokens=7, total_tokens=7)

In [21]:
queen_vector= np.array(queen_response.data[0].embedding)
print(queen_vector.shape)
print(queen_vector)

(3072,)
[ 0.01908419  0.00117847 -0.00398764 ...  0.00576628  0.00822409
  0.01082648]


In [22]:
# king_vector와 queen_vector의 유사도
king_queen_similarity = cosine_similarity(king_vector, queen_vector)
print("king과 queen의 유사도 :", king_queen_similarity)

king과 queen의 유사도 : 0.5006364215839533


In [ ]:
slave_response = openai_client.embeddings.create(
    input = "The slave begs",
    model = "text-embedding-3-large"
)

In [ ]:
slave_vector= np.array(slave_response.data[0].embedding)
print(slave_vector.shape)
print(slave_vector)

In [ ]:
king_slave_similarity = cosine_similarity(king_vector, slave_vector)
print("king문장과 slave문장 유사도:", king_slave_similarity)

In [ ]:
# 한국어 문장을 벡터로 바꿔도 유사도가 비슷

In [23]:
response = openai_client.embeddings.create(
    input="왕은 왕자의 아버지다",
    model="text-embedding-3-large"
)

In [24]:
response.data[0].embedding

[0.014605236239731312,
 0.00814149808138609,
 0.002996691968291998,
 -0.0034040091559290886,
 0.034680161625146866,
 0.020016737282276154,
 0.024962734431028366,
 -0.012141936458647251,
 0.007918443530797958,
 -0.012752912007272243,
 0.009470128454267979,
 0.017660116776823997,
 -0.021956345066428185,
 0.004361690022051334,
 0.026223478838801384,
 0.013819695450365543,
 -0.01389728020876646,
 0.02502092346549034,
 -0.024012327194213867,
 0.005581217352300882,
 -0.017310988157987595,
 -0.046007465571165085,
 0.020482243970036507,
 -0.008810661733150482,
 -0.0241674967110157,
 -0.0061873444356024265,
 0.0007400810718536377,
 0.009455581195652485,
 0.018494147807359695,
 -0.0037676854990422726,
 0.011133341118693352,
 0.030917325988411903,
 0.01580779254436493,
 0.018290488049387932,
 -0.02928805723786354,
 0.02189815603196621,
 -0.02457481250166893,
 -0.014653726480901241,
 -0.008820360526442528,
 -0.027193281799554825,
 -0.03357458859682083,
 -0.01666121929883957,
 -0.06226136535406113,

In [26]:
kor_king_vector = np.array(response.data[0].embedding)
print(kor_king_vector)

[0.01460524 0.0081415  0.00299669 ... 0.00721534 0.01149217 0.00849063]


In [27]:
response = openai_client.embeddings.create(
    input="여왕은 왕자의 어머니다",
    model="text-embedding-3-large"
)

In [29]:
kor_queen_vector = np.array(response.data[0].embedding)
print(kor_queen_vector)

[-0.00633463  0.01295298  0.00534164 ... -0.00638844 -0.00050108
  0.0337521 ]


In [30]:
print("왕문장과 여왕문장의 유사도 :", cosine_similarity(kor_king_vector, kor_queen_vector))

왕문장과 여왕문장의 유사도 : 0.5744286874279486


In [31]:
response = openai_client.embeddings.create(
    input="노예가 구걸한다",
    model="text-embedding-3-large"
)

In [33]:
kor_slave_vector = np.array(response.data[0].embedding)
print(kor_slave_vector)

[ 0.00521699  0.00297197  0.00265126 ... -0.01131553 -0.00267593
  0.01133527]


In [34]:
print("왕문장과 노예문장의 유사도 :", cosine_similarity(kor_king_vector, kor_slave_vector))

왕문장과 노예문장의 유사도 : 0.13551127309115693


In [35]:
# king_vector와 kor_king_vector의 유사도
print("king문장과 왕문장의 유사도 :", cosine_similarity(king_vector, kor_king_vector))

king문장과 왕문장의 유사도 : 0.5735245660442919


# 4. upstage의 embedding model 사용
- 한국어에서는 openai보다 성능이 좋다

In [36]:
upstage_client = OpenAI(
    api_key=upstage_key,
    base_url="https://api.upstage.ai/v1"
)

In [37]:
response = upstage_client.embeddings.create(
    input = "The king is prince's father",
    model="embedding-query"
)

In [39]:
up_king_vector = np.array(response.data[0].embedding)
print(up_king_vector.shape)

(4096,)


In [40]:
response = upstage_client.embeddings.create(
    input = "The queen is prince's mother",
    model="embedding-query"
)

In [42]:
up_queen_vector = np.array(response.data[0].embedding)
print(up_queen_vector.shape)

(4096,)


In [43]:
print("king문장과 queen문장의 유사도 :", cosine_similarity(up_king_vector, up_queen_vector))

king문장과 queen문장의 유사도 : 0.6695590665947906


In [44]:
response = upstage_client.embeddings.create(
    input = "왕은 왕자의 아버지다",
    model="embedding-query"
)

In [45]:
up_kor_king_vector = np.array(response.data[0].embedding)
print(up_kor_king_vector.shape)

(4096,)


In [46]:
response = upstage_client.embeddings.create(
    input = "여왕은 왕자의 어머니다",
    model="embedding-query"
)

In [47]:
up_kor_queen_vector = np.array(response.data[0].embedding)
print(up_kor_queen_vector.shape)

(4096,)


In [49]:
print("왕문장과 여왕문장의 유사도 :", cosine_similarity(up_kor_king_vector, up_kor_queen_vector))

왕문장과 여왕문장의 유사도 : 0.6502865998932696


In [51]:
print("king문장과 왕문장의 유사도 :", cosine_similarity(up_king_vector, up_kor_king_vector))

king문장과 왕문장의 유사도 : 0.8496098630487274


In [52]:
print("queen문장과 여왕문장의 유사도 :", cosine_similarity(up_queen_vector, up_kor_queen_vector))

queen문장과 여왕문장의 유사도 : 0.802447131719098
